#### Agent Executor with force function callign

We can make the agent use a tool at the beginning of its execution using langgraph. This is useful when you want to force agents to call particular tools, but still want flexibility of what happens after that

In [7]:
import os
from langchain import hub
from dotenv import load_dotenv
from langchain_core.agents import AgentAction, AgentFinish
from typing import TypedDict, Sequence, Annotated
import operator
import json

from langchain.agents.openai_functions_agent.base  import create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI
from langgraph.prebuilt import ToolExecutor
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_community.tools.tavily_search import TavilySearchResults

In [2]:
load_dotenv()

True

#### Define the tools that would be used by our agent and create the agent
In this first section we will be defining our agent to use the Tavily search tool, and we will be pulling the openai function agent prompt from langchain hub


In [5]:
tools = [TavilySearchResults(max_results=1), PythonREPLTool()]
llm = ChatOpenAI(model="mistralai/Mixtral-8x7B-Instruct-v0.1")
prompt = hub.pull("hwchase17/openai-functions-agent")

agent_runnable = create_openai_functions_agent(llm, tools=tools, prompt=prompt)
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

#### Define the graph state
In this section we will be defining the state of our langchain graph, the state is the value is passed across all the nodes in the graph,

In [10]:
from langchain_core.messages import BaseMessage
from typing import Union, Tuple

In [22]:
class AgentState(TypedDict):
    input: str
    chat_history: Annotated[Sequence[BaseMessage], operator.add]
    agent_outcome: Annotated[Union[AgentAction, AgentFinish, None], operator.setitem]
    intermediate_steps: Annotated[Sequence[Tuple[AgentAction, str]], operator.add]

##### Define the nodes
We now need to define a few different nodes in our graph. In `langgraph`, a node can be either a function or a runnable, There are two main noes we need for this
- The agent
- The function invocation tool

In [23]:
tool_executor = ToolExecutor(tools)


def run_agent(state: AgentState):
    agent_outcome = agent_runnable.invoke(state)
    return {"agent_outcome": agent_outcome}


def run_tool(state: AgentState):
    agent_action = state["agent_outcome"]
    result = tool_executor.invoke(agent_action)
    return {"intermediate_steps": [(agent_action, str(result))]}


def should_continue(state: AgentState):
    agent_outcome = state["agent_outcome"]
    
    if isinstance(agent_outcome, AgentFinish):
        return "end"
    
    else:
        return "contine"

In [24]:
from langchain_core.agents import AgentActionMessageLog


def first_agent(state: AgentState):
    action = AgentActionMessageLog(
        tool="tavily_search_results_json",
        tool_input=state["input"],
        log="",
        message_log=[]
    )
    return {"agent_outcome": action}
    

#### Define the graph
Now that we have individualy defined all the tools, agents, state and nodes, we can then put it all together into defining the langgraph graph

In [25]:
from langgraph.graph import StateGraph, END

In [26]:
workflow = StateGraph(AgentState)

workflow.add_node("agent", run_agent)
workflow.add_node("starting", first_agent)
workflow.add_node("action", run_tool)


workflow.add_edge("action", "agent")
workflow.set_entry_point("starting")
workflow.add_edge("starting", "action")
workflow.add_conditional_edges("agent", should_continue, {"continue": "action", "end": END})

app = workflow.compile()

#### Using the langgraph graph

Now we have the graph defined and compiled, we can now use it to perform inference

In [27]:
inputs = {
    "input": "what is the weather in sf?", 
    "chat_history": []
}

for output in app.stream(inputs):
    for key, value in output.items():
        print(f"output for node {key}")
        print("----")
        print(value)
    print("----")

output for node starting
----
{'agent_outcome': AgentActionMessageLog(tool='tavily_search_results_json', tool_input='what is the weather in sf?', log='', message_log=[])}
----
output for node action
----
{'intermediate_steps': [(AgentActionMessageLog(tool='tavily_search_results_json', tool_input='what is the weather in sf?', log='', message_log=[]), '[{\'url\': \'https://www.weatherapi.com/\', \'content\': "Weather in San Francisco is {\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1708084290, \'localtime\': \'2024-02-16 3:51\'}, \'current\': {\'last_updated_epoch\': 1708083900, \'last_updated\': \'2024-02-16 03:45\', \'temp_c\': 10.0, \'temp_f\': 50.0, \'is_day\': 0, \'condition\': {\'text\': \'Partly cloudy\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/116.png\', \'code\': 1003}, \'wind_mph\': 5.6, \'wind_kph\': 9.0, 